In [1]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


'\nPurpose: To decompose the multi-somas for splitting\nusing the new decomposition method\n\n\n\n'

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-12 04:16:25,280 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-12 04:16:25,281 - settings - Setting database.user to celiib
INFO - 2021-01-12 04:16:25,282 - settings - Setting database.password to newceliipass
INFO - 2021-01-12 04:16:25,286 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-12 04:16:25,287 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-12 04:16:25,298 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-12 04:16:25,580 - settings - Setting enable_python_native_blobs to True


In [3]:
test_mode = True

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-12 04:16:25,613 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-12 04:16:25,614 - settings - Setting database.user to celiib
INFO - 2021-01-12 04:16:25,615 - settings - Setting database.password to newceliipass
INFO - 2021-01-12 04:16:25,616 - settings - Setting enable_python_native_blobs to True


Sleeping 56 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-12 04:16:25,880 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [5]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [6]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 4110


In [7]:
#so that it will have the adapter defined
from datajoint_utils import *

In [8]:
decimation_version = 0
decimation_ratio = 0.25
key_source = (minnie.Decimation().proj(decimation_version='version')  & 
                  dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
                  & minnie.MultiSomaProofread2() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid()).proj()))
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134947393276,0,0.25
864691135012398582,0,0.25
864691135012471798,0,0.25
864691135012571126,0,0.25
864691135012905110,0,0.25
864691135065018692,0,0.25
864691135065024068,0,0.25
864691135065032772,0,0.25
864691135065033540,0,0.25
864691135081745143,0,0.25


In [9]:
#schema.external['decomposition'].delete(delete_external_files=True)

In [10]:
#minnie.Decomposition.drop()

In [11]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25
process_version = 0

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    process_version : int unsigned #the version of the preprocessing pipeline run
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """


    
    key_source = (minnie.Decimation().proj(decimation_version='version')  & 
                  dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
                  & minnie.MultiSomaProofread2() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid()).proj()))
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)
        

        #3) Get the somas info 
        somas = du.get_soma_mesh_list(segment_id) 
        soma_ver = du.get_soma_mesh_list_ver(segment_id)
                  
        
        print(f"somas = {somas}")
                  
        #3b) Get the glia and nuclei information 
        glia_faces,nuclei_faces = du.get_segment_glia_nuclei_faces(segment_id,return_empty_list=True)
        
                  
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"],
        glia_faces=glia_faces,
        nuclei_faces = nuclei_faces,

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       ver = soma_ver,
                       process_version = process_version,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [14]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,0dfc10119e45edcafbe0d8932a64f53f,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.166,at-node39,29,30409,2021-01-12 01:21:20
__decomposition,0e3ddd7deaf9c0a558c96067165b2a30,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,29,30386,2021-01-12 01:21:25
__decomposition,1071bede410fb82537727d706a6779da,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.145,at-node18,30,30264,2021-01-12 00:57:41
__decomposition,15334694f16f59d3b363e63519c951d4,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,29,30398,2021-01-12 01:18:44
__decomposition,159fb5a371bd94dd452794c391f31487,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.153,at-node26,30,30289,2021-01-12 00:31:41
__decomposition,1bb2b246f4acc8687ff62021801e8c0f,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.167,at-node40,29,30246,2021-01-12 00:21:05
__decomposition,1c7677a63d052676dc2a5d0e819b9425,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.170,at-node43,29,30243,2021-01-11 23:01:48
__decomposition,1dd6397125b48352b3a53e23b5447f81,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.172,at-node45,29,30275,2021-01-12 01:09:09
__decomposition,223292d754d01c10ec97cae17b086998,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.175,at-node48,29,30241,2021-01-11 22:34:53
__decomposition,2fce6f814c2680709bb844fc65ac2a5b,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.147,at-node20,29,30266,2021-01-12 01:00:20


In [13]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

INFO - 2021-01-12 04:16:26,069 - autopopulate - Found 143 keys to populate
INFO - 2021-01-12 04:16:26,077 - connection - Transaction started
INFO - 2021-01-12 04:16:26,079 - autopopulate - Populating: {'segment_id': 864691135012398582, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started



---- Working on Neuron 864691135012398582 ----
somas = [[<trimesh.Trimesh(vertices.shape=(21199, 3), faces.shape=(42025, 3))>, <trimesh.Trimesh(vertices.shape=(20673, 3), faces.shape=(41105, 3))>], array([689.7218, 689.7218]), array([0.888, 0.853])]

--- Beginning preprocessing of 864691135012398582---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(21199, 3), faces.shape=(42025, 3))>, <trimesh.Trimesh(vertices.shape=(20673, 3), faces.shape=(41105, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(21199, 3), faces.shape=(42025, 3))>, <trimesh.Trimesh(vertices.shape=(20673, 3), faces.shape=(41105, 3))>]
soma_mesh_list_centers = [array([1025739.12849663,  687951.7541535 ,  849333.87810274]), array([1036693.36598462,  690277.23206598,  863783.85420114])]
Getting Glia and Nuclei Pi

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:18<00:00, 18.39s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 22.29015827178955
branches_touching_root = [7]
length of Graph = 46020
Working on path [29693. 29697. 29710. 29718. 29725. 29731. 29740. 29744.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
Working on path [32252. 32263. 32269.]
path_degrees = [3, 2, 3]
Working on path [37052. 37088. 37129. 37175. 37204. 37228. 37252.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
After combining close endpoints max(kept_branches_idx) = 118, len(kept_branches_idx) = 116
empty_indices % = 0.0
 conflict_indices % = 0.005526729338986946


...


There was only one mesh found from the spine process and mesh split, returning empty array



Working on limb L0 branch 101
Working on limb L0 branch 81
Working on limb L0 branch 93
Working on limb L0 branch 5
Working on limb L0 branch 17
Working on limb L0 branch 1
Working on limb L0 branch 92
Working on limb L0 branch 0
Working on limb L0 branch 16
No spines and using precomputed width
Working on limb L0 branch 72
Working on limb L0 branch 95
Working on limb L0 branch 46
No spines and using precomputed width
Working on limb L0 branch 67
Working on limb L0 branch 90
Working on limb L0 branch 116
Working on limb L0 branch 61
Working on limb L0 branch 178
No spines and using precomputed width
Working on limb L0 branch 78
Working on limb L0 branch 89
Working on limb L0 branch 9
No spines and using precomputed width
Working on limb L0 branch 64
Working on limb L0 branch 54
Working on limb L0 branch 55
Working on limb L0 branch 26
Working on limb L0 branch 75
Working on limb L0 branch 82
Working on limb L0 branch 83
Working on limb L0 branch 2
No spines and using precomputed width

Working on limb L0 branch 11
Working on limb L0 branch 77
Working on limb L0 branch 112
No spines and using precomputed width
Working on limb L0 branch 129
Working on limb L0 branch 132
Working on limb L0 branch 156
Working on limb L0 branch 12
No spines and using precomputed width
Working on limb L0 branch 20
No spines and using precomputed width
Working on limb L0 branch 98
No spines and using precomputed width
Working on limb L0 branch 99
No spines and using precomputed width
Working on limb L0 branch 111
No spines and using precomputed width
Working on limb L0 branch 131
No spines and using precomputed width
Working on limb L0 branch 133
Working on limb L0 branch 4
Working on limb L0 branch 19
No spines and using precomputed width
Working on limb L0 branch 44
No spines and using precomputed width
Working on limb L0 branch 94
No spines and using precomputed width
Working on limb L0 branch 100
No spines and using precomputed width
Working on limb L0 branch 134
Working on limb L0 bran

Working on limb L2 branch 57
Working on limb L2 branch 44
Working on limb L2 branch 61
Working on limb L2 branch 0
Working on limb L2 branch 32
No spines and using precomputed width
Working on limb L2 branch 41
No spines and using precomputed width
Working on limb L2 branch 1
Working on limb L2 branch 50
Working on limb L2 branch 35
Working on limb L2 branch 36
No spines and using precomputed width
Working on limb L2 branch 64
No spines and using precomputed width
Working on limb L2 branch 65
No spines and using precomputed width
Working on limb L2 branch 22
No spines and using precomputed width
Working on limb L2 branch 30
No spines and using precomputed width
Working on limb L2 branch 26
No spines and using precomputed width
Working on limb L2 branch 38
No spines and using precomputed width
Working on limb L2 branch 2
Working on limb L2 branch 3
Working on limb L2 branch 4
Working on limb L2 branch 12
No spines and using precomputed width
Working on limb L2 branch 7
No spines and usi

Working on limb L10 branch 1
Working on limb L10 branch 3
No spines and using precomputed width
Working on limb L10 branch 12
No spines and using precomputed width
Working on limb L10 branch 14
No spines and using precomputed width
Working on limb L10 branch 0
No spines and using precomputed width
Working on limb L10 branch 13
No spines and using precomputed width
Working on limb L10 branch 2
No spines and using precomputed width
Working on limb L10 branch 4
No spines and using precomputed width
Working on limb L11 branch 0
No spines and using precomputed width
Working on limb L11 branch 1
No spines and using precomputed width
Working on limb L11 branch 2
No spines and using precomputed width
Working on limb L11 branch 3
No spines and using precomputed width
Working on limb L11 branch 4
No spines and using precomputed width
Working on limb L11 branch 5
Working on limb L11 branch 6
No spines and using precomputed width
Working on limb L11 branch 7
No spines and using precomputed width
W